In [41]:
import rltk
import csv
from datetime import datetime
tokenizer = rltk.tokenizer.crf_tokenizer.crf_tokenizer.CrfTokenizer()

In [42]:
class gCompanyRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['ID']
    
    # attribute 1: company name
    @rltk.cached_property
    def name_string(self):
        return self.raw_object['name'].lower()
    
    # attribute 2: headquarter
    @rltk.cached_property
    def headquarter_string(self):
        return self.raw_object['city']
    
    # attribute 3: company size
    @rltk.cached_property
    def size_string(self):
        return self.raw_object['size']
    
    # attribute 4：website
    @rltk.cached_property
    def website_string(self):
        website = self.raw_object['company_url']
        if website != "":
            if website[-1] != "/":
                website += "/" 
        return website
    
    # attribute 5: first three characters in company name
    @rltk.cached_property
    def first3title(self):
        chars = self.name_string[:3]
        return chars
    
    # attribute 6: website removed http
    @rltk.cached_property
    def website_nohttp(self):
        website = self.website_string
        website = website.replace("http://", "").replace("https://", "")
        return website
    
    
class lCompanyRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['ID']
    
    # attribute 1: company name
    @rltk.cached_property
    def name_string(self):
        return self.raw_object['company_name'].lower()
    
    # attribute 2: headquarter
    @rltk.cached_property
    def headquarter_string(self):
        return self.raw_object['city']
    
    # attribute 3: company size
    @rltk.cached_property
    def size_string(self):
        return self.raw_object['size']
    
    # attribute 4：website
    @rltk.cached_property
    def website_string(self):
        website = self.raw_object['website']
        if website != "":
            if website[-1] != "/":
                website += "/" 
        return website
    
    # attribute 5: first three characters in company name
    @rltk.cached_property
    def first3title(self):
        chars = self.name_string[:3]
        return chars
    
    # attribute 6: website removed http
    @rltk.cached_property
    def website_nohttp(self):
        website = self.website_string
        website = website.replace("http://", "").replace("https://", "")
        return website

class wCompanyRecord(rltk.Record):
    def __init__(self, raw_object):
        super().__init__(raw_object)
        self.name = ''

    @rltk.cached_property
    def id(self):
        return self.raw_object['item']
    # attribute 1: company name
    @rltk.cached_property
    def name_string(self):
        return self.raw_object['itemLabel'].lower()
    
    # attribute 2: headquarter
    @rltk.cached_property
    def headquarter_string(self):
        return self.raw_object['locationLabel']
    
    # attribute 4：website
    @rltk.cached_property
    def website_string(self):
        website = self.raw_object['website']
        if website != "":
            if website[-1] != "/":
                website += "/" 
        return website
    
    # attribute 5: first three characters in company name
    @rltk.cached_property
    def first3title(self):
        chars = self.name_string[:3]
        return chars
    
    # attribute 6: website removed http
    @rltk.cached_property
    def website_nohttp(self):
        website = self.website_string
        website = website.replace("http://", "").replace("https://", "")
        return website

In [43]:
dir_ = '../csvfile_category/'
gc_file = dir_ + 'glassdoor_company_withid.csv'
lc_file = dir_ + 'linkedin_company_withid.csv'
wc_file = dir_ + 'wikidata_company.csv'

ds_linkedin = rltk.Dataset(rltk.CSVReader(lc_file),record_class=lCompanyRecord)
ds_glassdoor = rltk.Dataset(rltk.CSVReader(gc_file),record_class=gCompanyRecord)
ds_wikidata = rltk.Dataset(rltk.CSVReader(wc_file),record_class=wCompanyRecord)


### blocking linkedin-glassdoor

In [44]:
bg = rltk.HashBlockGenerator()

block = bg.generate(
    bg.block(ds_glassdoor, property_="first3title"),
    bg.block(ds_linkedin, property_="first3title")
)

### entity linking

In [45]:
def title_string_similarity(s1, s2):
    return rltk.jaro_winkler_similarity(s1, s2)
    
def name_string_similarity(s1, s2):
    return 1-rltk.levenshtein_distance(s1, s2)

def website_similarity(s1, s2):
    if s1 == s2:
        return 1
    if s1 == "" or s2 == "":
        return 0.5
    return 0

In [46]:
# threshold value to determine if we are confident the record match
MY_TRESH = 0.7

# entity linkage scoring function
def rule_based_method(r1, r2):
    score_1 = name_string_similarity(r1.name_string, r2.name_string)
    score_2 = website_similarity(r1.website_string, r2.website_string)
    
    total = 0.7 * score_1 + 0.3 * score_2
    
    # return two values: boolean if they match or not, float to determine confidence
    return total > MY_TRESH, total

In [47]:
matchPairs = []
for r_glassdoor, r_linkedin in rltk.get_record_pairs(ds_glassdoor, ds_linkedin, block=block):
    result, confidence = rule_based_method(r_glassdoor, r_linkedin)
    if result == True:
        matchPairs.append([r_glassdoor.id, r_linkedin.id, confidence])

In [48]:
len(matchPairs)

64

In [49]:
for ids in matchPairs:
    print(ids)
    r_glassdoor = ds_glassdoor.get_record(ids[0])
    r_linkedin = ds_linkedin.get_record(ids[1])
    print(r_glassdoor.name_string, r_glassdoor.website_string)
    print(r_linkedin.name_string, r_linkedin.website_string, "\n")

['5', '2434', 1.0]
the home depot http://www.careers.homedepot.com/
the home depot http://www.careers.homedepot.com/ 

['6', '2633', 1.0]
usaa http://www.usaajobs.com/
usaa http://www.usaajobs.com/ 

['18', '1168', 1.0]
bp http://www.bp.com/
bp http://www.bp.com/ 

['592', '1756', 1.0]
citi http://www.citigroup.com/
citi http://www.citigroup.com/ 

['739', '1944', 1.0]
capstoneone search http://www.capstoneonesearch.com/
capstoneone search http://www.capstoneonesearch.com/ 

['765', '2229', 1.0]
expedition technology inc http://www.exptechinc.com/
expedition technology inc http://www.exptechinc.com/ 

['25', '2314', 1.0]
liberty mutual insurance http://www.libertymutualgroup.com/
liberty mutual insurance http://www.libertymutualgroup.com/ 

['29', '1926', 1.0]
ursus, inc. http://www.ursusinc.com/
ursus, inc. http://www.ursusinc.com/ 

['30', '69', 1.0]
harnham http://www.harnham.com/
harnham http://www.harnham.com/ 

['31', '2501', 1.0]
apex systems http://www.apexsystems.com/
apex sys

In [50]:
with open(dir_ + 'linkage/g_l_linkage.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["glassdoor.id", "linkedin.id"])
    for row in matchPairs:
        writer.writerow((row[0], row[1]))

### blocking linkedin-wikidata

In [51]:
bg2 = rltk.HashBlockGenerator()

block2 = bg2.generate(
    bg2.block(ds_wikidata, property_="first3title"),
    bg2.block(ds_linkedin, property_="first3title")
)

In [52]:
def title_string_similarity(s1, s2):
    return rltk.jaro_winkler_similarity(s1, s2)
def website_similarity2(s1, s2):
    if s1.lower() == s2.lower() and s1 != "" and s2 != "":
        return 1
    if s1 == "" and s2 == "":
        return 0.05
    if s1 == "":
        return 0.3
    if s2 == "":
        return 0.1
    return 0

In [53]:
MY_TRESH = 0.759

# entity linkage scoring function
def rule_based_method2(r1, r2):
    score_1 = title_string_similarity(r1.name_string, r2.name_string)
    score_2 = website_similarity2(r1.website_nohttp, r2.website_nohttp)
    if r1.name_string == "Uber Technology" and r2.name_string == "Uber":
        print(r1.website_string, r2.website_string)
        print(r1.website_nohttp, r2.website_nohttp)
        print(score_1, score_2, 0.7*score_1 + 0.3*score_2)
    total = 0.76*score_1 + 0.24*score_2
    
    return total > MY_TRESH, total, score_1, score_2

In [54]:
matchPairs_wl = []
for r_wikidata, r_linkedin in rltk.get_record_pairs(ds_wikidata, ds_linkedin, block=block2):
    result, confidence, score_1, score_2 = rule_based_method2(r_wikidata, r_linkedin)
    if result == True:
        matchPairs_wl.append([r_wikidata.id, r_linkedin.id, confidence, score_1, score_2])

In [55]:
len(matchPairs_wl)

567

In [28]:
for ids in matchPairs_wl:
    print(ids)
    r_wikidata = ds_wikidata.get_record(ids[0])
    r_linkedin = ds_linkedin.get_record(ids[1])
    print(r_wikidata.website_string, r_wikidata.name_string)
    print(r_linkedin.website_string, r_linkedin.name_string, "\n")

['http://www.wikidata.org/entity/Q4633233', '106', 0.76, 1.0, 0]
http://www.2u.com/ 2u
http://2u.com/ 2u 

['http://www.wikidata.org/entity/Q259340', '1001', 1.0, 1.0, 1]
https://www.7-eleven.com/ 7-eleven
http://www.7-ELEVEn.com/ 7-eleven 

['http://www.wikidata.org/entity/Q4658862', '1027', 1.0, 1.0, 1]
http://www.aplaceformom.com/ a place for mom
http://www.aplaceformom.com/ a place for mom 

['http://www.wikidata.org/entity/Q306764', '2625', 0.896, 0.8631578947368421, 1]
http://www.abbott.com/ abbott laboratories
https://www.abbott.com/ abbott 

['http://www.wikidata.org/entity/Q14662364', '942', 1.0, 1.0, 1]
http://www.abbvie.com/ abbvie
http://www.abbvie.com/ abbvie 

['http://www.wikidata.org/entity/Q104290524', '2625', 0.8986666666666666, 0.8666666666666666, 1]
https://www.abbott.com/ abbott diagnostics
https://www.abbott.com/ abbott 

['http://www.wikidata.org/entity/Q4672981', '31', 0.924, 0.9, 1]
https://www.acehardware.com/ ace hardware
http://www.acehardware.com/ ace hardw

In [29]:
# deduplicate
matchPairs_wl1 = {}
# depulicate wikidata id:
for ids in matchPairs_wl:
    if ids[0] not in matchPairs_wl1:
        matchPairs_wl1[ids[0]] = []
    matchPairs_wl1[ids[0]].append(ids)
for i in matchPairs_wl1:
    if len(matchPairs_wl1[i]) > 1:
        best = max(matchPairs_wl1[i], key=lambda x: x[2])
        matchPairs_wl1[i] = best
    else:
        matchPairs_wl1[i] = matchPairs_wl1[i][0]

# depulicate glassdoor id:
matchPairs_wl2 = {}
for ids in matchPairs_wl1.values():
    if ids[1] not in matchPairs_wl2:
        matchPairs_wl2[ids[1]] = []
    matchPairs_wl2[ids[1]].append(ids)

for i in matchPairs_wl2:
    if len(matchPairs_wl2[i]) > 1:
        best = max(matchPairs_wl2[i], key=lambda x: x[2])
        matchPairs_wl2[i] = best
    else:
        matchPairs_wl2[i] = matchPairs_wl2[i][0]
matchPairs_wl_deduplicate = list(matchPairs_wl2.values())
len(matchPairs_wl_deduplicate)

515

In [30]:
with open(dir_ + 'linkage/w_l_linkage.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["wikidata.id", "linkedin.id"])
    for row in matchPairs_wl_deduplicate:
        writer.writerow((row[0], row[1]))

### blocking linkedin-wikidata

In [31]:
bg3 = rltk.HashBlockGenerator()

block3 = bg3.generate(
    bg3.block(ds_wikidata, property_="first3title"),
    bg3.block(ds_glassdoor, property_="first3title")
)

In [32]:
matchPairs_wg = []
for r_wikidata, r_glassdoor in rltk.get_record_pairs(ds_wikidata, ds_glassdoor, block=block3):
    result, confidence, score_1, score_2 = rule_based_method2(r_wikidata, r_glassdoor)
    if result == True:
        matchPairs_wg.append([r_wikidata.id, r_glassdoor.id, confidence, score_1, score_2])

In [33]:
len(matchPairs_wg)

163

In [34]:
for ids in matchPairs_wg:
    print(ids)
    r_wikidata = ds_wikidata.get_record(ids[0])
    r_glassdoor = ds_glassdoor.get_record(ids[1])
    print(r_wikidata.website_string, r_wikidata.name_string)
    print(r_glassdoor.website_string, r_glassdoor.name_string, "\n")

['http://www.wikidata.org/entity/Q4652197', '482', 0.8098333333333333, 0.9708333333333333, 0.3]
 aks inc
http://www.aksincorporated.com/ aks, inc 

['http://www.wikidata.org/entity/Q4733393', '129', 0.8685079365079365, 0.8269841269841269, 1]
http://www.ally.com/ ally bank
http://www.ally.com/ ally financial 

['http://www.wikidata.org/entity/Q2032800', '129', 1.0, 1.0, 1]
https://www.ally.com/ ally financial
http://www.ally.com/ ally financial 

['http://www.wikidata.org/entity/Q2645636', '127', 1.0, 1.0, 1]
https://www.allstate.com/ allstate
http://www.allstate.com/ allstate 

['http://www.wikidata.org/entity/Q344839', '456', 0.76, 1.0, 0]
http://www.acushnet.com/ acushnet company
http://www.acushnetholdingscorp.com/ acushnet company 

['http://www.wikidata.org/entity/Q11463', '46', 1.0, 1.0, 1]
https://www.adobe.com/ adobe
http://www.adobe.com/ adobe 

['http://www.wikidata.org/entity/Q30338375', '330', 0.8841904761904762, 0.8476190476190476, 1]
http://www.aecom.com/ aecom (united st

In [35]:
# deduplicate
matchPairs_wg1 = {}
# depulicate wikidata id:
for ids in matchPairs_wg:
    if ids[0] not in matchPairs_wg1:
        matchPairs_wg1[ids[0]] = []
    matchPairs_wg1[ids[0]].append(ids)
for i in matchPairs_wg1:
    if len(matchPairs_wg1[i]) > 1:
        best = max(matchPairs_wg1[i], key=lambda x: x[2])
        matchPairs_wg1[i] = best
    else:
        matchPairs_wg1[i] = matchPairs_wg1[i][0]

# depulicate glassdoor id:
matchPairs_wg2 = {}
for ids in matchPairs_wg1.values():
    if ids[1] not in matchPairs_wg2:
        matchPairs_wg2[ids[1]] = []
    matchPairs_wg2[ids[1]].append(ids)

for i in matchPairs_wg2:
    if len(matchPairs_wg2[i]) > 1:
        best = max(matchPairs_wg2[i], key=lambda x: x[2])
        matchPairs_wg2[i] = best
    else:
        matchPairs_wg2[i] = matchPairs_wg2[i][0]
matchPairs_wg_deduplicate = list(matchPairs_wg2.values())
len(matchPairs_wg_deduplicate)

145

In [36]:
with open(dir_ + 'linkage/w_g_linkage.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["wikidata.id", "glassdoor.id"])
    for row in matchPairs_wg_deduplicate:
        writer.writerow((row[0], row[1]))

In [37]:
def findID(id1, lst1):
    for i in lst1:
        if i[0] == id1:
            return i[1]
    return None

def findID2(id1, lst1):
    for i in lst1:
        if i[1] == id1:
            return i[0]
    return None

In [38]:
# combine three
# glassdoor - linkedin - wikidata
matchPairs_3 = []
id_gs = [i[0] for i in matchPairs]
for id_wg in matchPairs_wg_deduplicate:
    id_w, id_g = id_wg[0], id_wg[1]
    if id_g in id_gs:
        id_l = findID(id_g, matchPairs)
        matchPairs_3.append([id_g, id_l, id_w])

id_ls = [i[1] for i in matchPairs]
id_ls2 = [i[1] for i in matchPairs_3]
for id_wl in matchPairs_wl_deduplicate:
    id_w, id_l = id_wl[0], id_wl[1]
    if id_l in id_ls and id_l not in id_ls2:
        id_g = findID2(id_l, matchPairs)
        matchPairs_3.append([id_g, id_l, id_w])
        

In [39]:
matchPairs_3

[['52', '1858', 'http://www.wikidata.org/entity/Q484930'],
 ['487', '1237', 'http://www.wikidata.org/entity/Q23018218'],
 ['592', '1756', 'http://www.wikidata.org/entity/Q219508'],
 ['95', '1107', 'http://www.wikidata.org/entity/Q23016736'],
 ['921', '1022', 'http://www.wikidata.org/entity/Q5272601'],
 ['550', '1340', 'http://www.wikidata.org/entity/Q15589464'],
 ['377', '1576', 'http://www.wikidata.org/entity/Q1538585'],
 ['25', '2314', 'http://www.wikidata.org/entity/Q1516450'],
 ['736', '2472', 'http://www.wikidata.org/entity/Q634815'],
 ['102', '1673', 'http://www.wikidata.org/entity/Q6982632'],
 ['60', '2388', 'http://www.wikidata.org/entity/Q108443888'],
 ['5', '2434', 'http://www.wikidata.org/entity/Q864407'],
 ['6', '2633', 'http://www.wikidata.org/entity/Q7865722'],
 ['779', '2358', 'http://www.wikidata.org/entity/Q780442']]

In [67]:
with open(dir_ + 'linkage/g_l_w_linkage.csv', mode='w') as file:
    writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["glassdoor.id", "linkedin.id", "wikidata.id"])
    for row in matchPairs_3:
        writer.writerow((row[0], row[1], row[2]))